In [1]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util

In [2]:
df = pd.read_csv('diga_translated.csv')
df.head()

,app_name,developer_info,website_info,app_status,start_date,end_date,app_description,indication_codes,indication_names,price,property_values,price_averaged,indication_names_translated,app_description_translated
0,CANKADO PRO-React Onco,"CANKADO GmbH, Deutschland",partners.cankado.com/about/,vorläufig aufgenommen,03.05.2021,02.05.2023,PRO-React Onco ist eine Web- und App-basierte ...,C50,Bösartige Neubildung der Brustdrüse Mamma,399.84,"Keine Zusatzgeräte, Vertragsärztliche Leistung...",399.84,Malignant neuformation of the mamma breast,PRO-React Onco is a web- and app-based digital...
1,Cara Care für Reizdarm,"HiDoc Technologies GmbH, Deutschland",cara.care/de/about/,vorläufig aufgenommen,26.12.2021,25.11.2023,Cara Care für Reizdarm ist eine digitale Gesun...,"K58, K58.1, K58.2, K58.3, K58.8","Reizdarmsyndrom, Reizdarmsyndrom, Diarrhoe-prä...",718.20,"Keine Zusatzgeräte, Keine vertragsärztlichen L...",718.20,"Irritable bowel syndrome, Irritable bowel synd...",Cara Care for Irritable Intestinal is a digita...
2,companion patella powered by medi - proved by ...,"PrehApp GmbH, Deutschland",prehapp.de,vorläufig aufgenommen,04.10.2021,03.03.2023,companion patella powered by medi - proved by ...,"M22.2, M22.4, M76.5, M79.66, S83.0","Krankheiten im Patellofemoralbereich, Chondrom...",345.10,"Keine Zusatzgeräte, Vertragsärztliche Leistung...",345.10,"Patellofemoral diseases, chondromalacia patell...",companion patella powered by medi - proven by ...
3,deprexis,"GAIA AG, Deutschland",gaia-group.com/de/,Dauerhaft aufgenommen,NaN,NaN,deprexis ist ein interaktives onlinebasiertes ...,"F32.0, F32.1, F32.2, F33.0, F33.1, F33.2","Leichte depressive Episode, Mittelgradige depr...",210.00,"Keine Zusatzgeräte, Keine vertragsärztlichen L...",210.00,"Mild depressive episode, moderate depressive e...",deprexis is an interactive online self-help pr...
4,edupression.com®,"SOFY GmbH, Österreich",edupression.com/,vorläufig aufgenommen,26.12.2022,25.08.2023,Die DiGA edupression.com® besteht aus dem Medi...,"F32.0, F32.1, F33.0, F33.1","Leichte depressive Episode, Mittelgradige depr...",178.50 - 357.00,"Keine Zusatzgeräte, Keine vertragsärztlichen L...",267.75,"Mild depressive episode, intermediate depressi...",The DipdfGA edupression.com® consists of the M...


In [35]:
# take each row with app_name, developer_info, indication_codes, indication_names, app_description
df_ger = df[['app_name', 'developer_info', 'indication_codes', 'indication_names', 'app_description']]

# create a dictionary with row index as key and all the other columns as values as a single value
df_dict = df_ger.T.to_dict('list')
# do the same with the english version
df_en = df [['app_name', 'developer_info', 'indication_codes', 'indication_names_translated', 'app_description_translated']]

# create a dictionary with row index as key and all the other columns as values as a single value
df_dict_en = df_en.T.to_dict('list')

In [37]:
# join all values in to a single string
df_dict = {k: ' '.join(v) for k, v in df_dict.items()}
df_dict_en = {k: ' '.join(v) for k, v in df_dict_en.items()}

In [16]:
model1 = SentenceTransformer("sentence-transformers/msmarco-distilbert-dot-v5") #english
model2 = SentenceTransformer("Sahajtomar/German-semantic") #german

In [58]:
entries_ger = model2.encode(list(df_dict.values()))
entries_eng = model1.encode(list(df_dict_en.values()))

In [53]:
def german_top(input_text, dictionary, entries = entries_ger):
    input_embedding = model2.encode([input_text])
    similarity2 = np.dot(input_embedding, entries.T)
    # only return similarity scores above 120
    # similarity2 = np.where(similarity2 > 120)
    # print(similarity2)
    # if similarity2[0].max() == 0:
    #     print('No results found')
    #     return
    indices = np.argsort(similarity2)[0][-3:][::-1]
    # print(f'sorted top 5 indices: {indices}')
    # print(f'similarity scores: {similarity2[0][indices]}')
    return df_ger.iloc[indices]

In [66]:
input = 'Ich will endlich wieder richtig Sex haben koennen.'
german_top(input, df_dict)

,app_name,developer_info,indication_codes,indication_names,app_description
13,HelloBetter Vaginismus Plus,GET.ON Institut für Online Gesundheitstraining...,"F52.5, F52.6","Nichtorganischer Vaginismus, Nichtorganische D...",HelloBetter Vaginismus Plus ist eine digitale ...
17,Kranus Edera,"Kranus Health GmbH, Deutschland",N48.4,Impotenz organischen Ursprungs,Kranus Edera ist eine digitale Gesundheitsanwe...
22,NichtraucherHelden-App,"NichtraucherHelden GmbH, Deutschland",F17.2,Psychische und Verhaltensstörungen durch Tabak...,Die App dient der Behandlung und Linderung ein...


In [61]:
def english_top3(input_text, dictionary, entries = entries_eng):
    input_embedding = model1.encode([input_text])
    similarity2 = np.dot(input_embedding, entries.T)
    indices = np.argsort(similarity2, )[0][-3:][::-1]
    return df_en.iloc[indices]

In [64]:
input_eng = 'I want to fuck again.'
english_top3(input_eng, df_dict_en)

,app_name,developer_info,indication_codes,indication_names_translated,app_description_translated
17,Kranus Edera,"Kranus Health GmbH, Deutschland",N48.4,Impotence of organic origin,Kranus Edera is a digital health application f...
13,HelloBetter Vaginismus Plus,GET.ON Institut für Online Gesundheitstraining...,"F52.5, F52.6","Non-organic vaginism, Non-organic dyspareunia",HelloBetter Vaginismus Plus is a digital healt...
19,Meine Tinnitus App - Das digitale Tinnitus Cou...,"Sonormed GmbH, Deutschland",H93.1,Tinnitus aurium,My Tinnitus App - The digital Tinnitus Counsel...
